In [6]:
import pandas as pd
import numpy as np
from numpy import delete
import scipy.stats as stats
import time

Qué hay en este fichero:

- 0) Obtener los datos cada 15 mins de policromos
- 0.2) Separar datos por semanas

- 1) ¿Las oscilaciones diarias se deben a un intercambio de aire con el exterior? Coeficientes de correlación entre la diferencia de temperaturas entre exterior y cada sala con la cantidad de CO2. Variables sin suavizar
- 1.2) ¿Las oscilaciones diarias se deben a un intercambio de aire con el exterior? Coeficientes de correlación entre la diferencia de temperaturas entre exterior y cada sala con la cantidad de CO2. Variables suavizadas
- 2) ¿Hay relación entre la cantidad de CO2 que hay en cada sala? Coef correl CO2c-CO2h y CO2c-CO2p con las variables sin suavizar
- 2.2) ¿Hay relación entre la cantidad de CO2 que hay en cada sala? Coef correl CO2c-CO2h y CO2c-CO2p con las variables suavizadas

Solo obtener para los mese con oscilaciones claras:
- 1) Los meses con oscilaciones claras para el C02 del cruce son: 
 Julio, agosto y septiembre de 2021.
 Junio, julio, agosto y septiembre de 2022
- 2) Los meses con oscilaciones claras para el CO2 del hall son:
Julio, agosto, septiembre y octubre de 2021. Mayo, junio y julio de 2022

Compilar antes 0 y 0.1 cada vez que se vayan a obtener unos coeficientes para no perder datos

# 0) Obtener los datos cada 15 mins de policromos:
Necesario para obtener coef. de correl que usen datos de policromos

In [2]:
def adaptar_len_policromos(FicheroCruce, FicheroExterior, FicheroHall, FicheroPolicromos, mesAño):
    """
    Atributos:
    1) FicheroCruce: Excel con datos del cruce
    2) FicheroExterior: Excel con datos del exterior
    3) FicheroHall: Excel con datos del hall
    4) FicheroPolicromos: Excel con datos de policromos
    5) mesAño: String que indique el mes y año al que pertenecen los datos 
    
    Devuelve:
    1) datosPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado; Una lista de listas, donde cada elemento (Tp, Tp_suavizada, ect.)
    contiene los datos cada 15 minutos de policromos de la variable correspondiente.
    
    Cosas a tener en cuenta:
    1) No se tiene en cuenta una diferencia temporal entre los datos iniciales de datospolicromos con Ficheroref porque 
    policromos_cada15min ya lo hace. (datospolicromos se obtiene a partir de llamar a policromos_cada15min para un fichero de 
    policromos)
    2) Pinta un mensaje de aviso en caso de que la longitud de policromos sea diferente a la del resto de ficheros 
    """
#     print('---------------------------------------------------------------------------------------------------------------')
#     print('---------------------------------------------------------------------------------------------------------------')
#     print(mesAño)
    #Leer ficheros
    Cruce = pd.read_excel(FicheroCruce)
    Exterior = pd.read_excel(FicheroExterior)
    FicheroHall = pd.read_excel(FicheroHall)
    Policromos = pd.read_excel(FicheroPolicromos)
    
    #Copiar los datos cada 15 mins---------------------------------------------------------------------------------
    a=0
    b=len(Policromos['Fecha/Tiempo'])
    min1 =FicheroHall['Fecha/Tiempo'][0].minute #Minutos del 1er dato del FAdicional
    Tp=[]#El 0 es un número entero, coge desde el 1er dato útil
    Tp_suavizada=[]
    CO2=[]
    CO2_suavizado=[]
    Fecha_Tiempo = []
    while a<b-1:
        division=(Policromos['Fecha/Tiempo'][a].minute-min1)/15
        division_entera=(Policromos['Fecha/Tiempo'][a].minute-min1)//15
        if  division == division_entera and division>=0:
#             print(Policromos['Temperatura C (C)'][a], Policromos['Tp_suavizada'][a], Policromos['CO2 C (ppm)'][a], 
#                   Policromos['CO2p_suavizado'][a], Policromos['Fecha/Tiempo'][a], 'dato a copiar')
            Tp.append(Policromos['Temperatura C (C)'][a])
            Tp_suavizada.append(Policromos['Tp_suavizada'][a])
            CO2.append(Policromos['CO2 C (ppm)'][a])
            CO2_suavizado.append(Policromos['CO2p_suavizado'][a])
            Fecha_Tiempo.append(Policromos['Fecha/Tiempo'][a])
#             print(Tp[len(Tp)-1], Tp_suavizada[len(Tp_suavizada)-1], CO2[len(CO2)-1], CO2_suavizado[len(CO2_suavizado)-1],
#                   Fecha_Tiempo[len(Fecha_Tiempo)-1], 'dato copiado')
        a=a+1
    # Tenemos listas de Tp, Tp_suavizada, CO2, CO2_suavizado y Fecha/Tiempo cada 15 mins 
    
    #recortar ficheros-----------------------------------------------------------------------------------------------------
    if len(FicheroHall['Fecha/Tiempo'])< len(Tp):
        Tp=Tp[:len(FicheroHall['Fecha/Tiempo'])]
        Tp_suavizada=Tp_suavizada[:len(FicheroHall['Fecha/Tiempo'])]
        CO2=CO2[:len(FicheroHall['Fecha/Tiempo'])]
        CO2_suavizado=CO2_suavizado[:len(FicheroHall['Fecha/Tiempo'])]
        Fecha_Tiempo=Fecha_Tiempo[:len(FicheroHall['Fecha/Tiempo'])]
#      int('Policromos es más corto-> Solucionar')   
    #pintar datos-----------------------------------------------------------------------------------------------------------
#     print('Datos finales-------------------')
#     print('lenCruce:', 'lenExterior:', 'lenHall:', 'lenPolicromos:')
#     print(len(Cruce['Fecha/Tiempo']), len(Exterior['Fecha/Tiempo']), len(FicheroHall['Fecha/Tiempo']), len(Tp))
#     print('--------------------------------')
#     print('1er dato Cruce:', '1er dato Exterior:', '1er dato hall:', '1er dato Policromos:')
#     print(Cruce['Fecha/Tiempo'][0], Exterior['Fecha/Tiempo'][0],FicheroHall['Fecha/Tiempo'][0],Fecha_Tiempo[0])
#     print('--------------------------------')
#     print('Último dato Cruce:', 'Último dato Exterior:', 'Último dato hall:', 'Último dato Policromos:')
#     print(Cruce['Fecha/Tiempo'][len(Cruce['Fecha/Tiempo'])-1], Exterior['Fecha/Tiempo'][len(Exterior['Fecha/Tiempo'])-1],
#           FicheroHall['Fecha/Tiempo'][len(FicheroHall['Fecha/Tiempo'])-1],Fecha_Tiempo[len(Tp)-1])
    return(Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo)

In [3]:
#Datos cada 15 mins en policromos:
# 2020
DatosPolicromos_10_20 = adaptar_len_policromos("./altamira/2020/10.OCTUBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA ENTRADA.xlsx", 
                                                              "./altamira/2020/10.OCTUBRE/ZONA POLICROMOS.xlsx",'Octubre 2020')

DatosPolicromos_11_20 = adaptar_len_policromos("./altamira/2020/11.NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA POLICROMOS C.xlsx", 'Noviembre 2020')
DatosPolicromos_12_20 = adaptar_len_policromos("./altamira/2020/12.DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA POLICROMOS C.xlsx", 'Diciembre 2020')
#2021
DatosPolicromos_1_21 = adaptar_len_policromos("./altamira/2021/1.ENERO/ZONA CRUCE ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA EXTERIOR ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA ENTRADA ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA POLICROMOS ENERO.xlsx",'Enero 2021')
DatosPolicromos_2_21 = adaptar_len_policromos("./altamira/2021/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA POLICROMOS C.xlsx", 'Febrero 2021')
DatosPolicromos_3_21 = adaptar_len_policromos("./altamira/2021/3.MARZO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA POLICROMOS C.xlsx", 'Marzo 2021')
DatosPolicromos_4_21 = adaptar_len_policromos("./altamira/2021/4.ABRIL/ZONA CRUCE.xlsx",
                                                          "./altamira/2021/4.ABRIL/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/4.ABRIL/ZONA ENTRADA.xlsx",
                                                          "./altamira/2021/4.ABRIL/ZONA POLICROMOS C.xlsx", 'Abril 2021')
DatosPolicromos_5_21 = adaptar_len_policromos("./altamira/2021/5.MAYO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA POLICROMOS C.xlsx", 'Mayo 2021')
DatosPolicromos_6_21 = adaptar_len_policromos("./altamira/2021/6.JUNIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/6.JUNIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/6.JUNIO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/6.JUNIO/ZONA POLICROMOS C.xlsx", 'Junio 2021')
DatosPolicromos_7_21 = adaptar_len_policromos("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/7.JULIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/7.JULIO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/7.JULIO/ZONA POLICROMOS C.xlsx", 'Julio 2021')
DatosPolicromos_8_21 = adaptar_len_policromos("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/8.AGOSTO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/8.AGOSTO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/8.AGOSTO/ZONA POLICROMOS C.xlsx", 'Agosto 2021')
DatosPolicromos_9_21 = adaptar_len_policromos("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA POLICROMOS C.xlsx", 'Septiembre 2021')
DatosPolicromos_10_21 = adaptar_len_policromos("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA POLICROMOS.xlsx", 'Octubre 2021')
DatosPolicromos_11_21 = adaptar_len_policromos("./altamira/2021/11. NOVIEMBRE/ZONA CRUCE D.xlsx", 
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA POLICROMOS.xlsx", 'Noviembre 2021')
DatosPolicromos_12_21 = adaptar_len_policromos("./altamira/2021/12. DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2021/12. DICIEMBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2021/12. DICIEMBRE/ZONA ENTRADA.xlsx",
                                                              "./altamira/2021/12. DICIEMBRE/ZONA POLICROMOS.xlsx", 'Diciembre 2021')
#2022
DatosPolicromos_1_22 = adaptar_len_policromos("./altamira/2022/1.ENERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/1.ENERO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/1.ENERO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/1.ENERO/ZONA POLICROMOS C.xlsx", 'Enero 2022')
DatosPolicromos_2_22 = adaptar_len_policromos("./altamira/2022/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA POLICROMOS C.xlsx", 'Febrero 2022')
DatosPolicromos_3_22 = adaptar_len_policromos("./altamira/2022/3.MARZO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/3.MARZO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/3.MARZO/ZONA POLICROMOS C.xlsx", 'Marzo 2022')
DatosPolicromos_4_22 = adaptar_len_policromos("./altamira/2022/4.ABRIL/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA POLICROMOS C.xlsx", 'Abril 2022')
#mayo 22 cruce vacío- A posteriori me hizo falta suavizar exterior
#Para poder utilizar la función suavizar poner otro archivo de cruce cualquiera.
DatosPolicromos_5_22 = adaptar_len_policromos("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/5.MAYO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/5.MAYO/ZONA ENTRADA.xlsx",
                                                          "./altamira/2022/5.MAYO/ZONA POLICROMOS.xlsx", 'Mayo 2022')
DatosPolicromos_6_22 = adaptar_len_policromos("./altamira/2022/6. JUNIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/6. JUNIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/6. JUNIO/ZONA ENTRADA.xlsx",
                                                          "./altamira/2022/6. JUNIO/ZONA POLICROMOS C.xlsx", 'Junio 2022')
DatosPolicromos_7_22 = adaptar_len_policromos("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
                                                          "./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/7.JULIO/ZONA POLICROMOS C.xlsx", 'Julio 2022')
DatosPolicromos_8_22 = adaptar_len_policromos("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/8. AGOSTO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/8. AGOSTO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/8. AGOSTO/ZONA POLICROMOS C.xlsx", 'Agosto 2022')
DatosPolicromos_9_22 = adaptar_len_policromos("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA POLICROMOS C.xlsx", 'Septiembre 2022')
DatosPolicromos_10_22 = adaptar_len_policromos("./altamira/2022/10. OCTUBRE/ZONA CRUCE D.xlsx", 
                                                              "./altamira/2022/10. OCTUBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2022/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2022/10. OCTUBRE/ZONA POLICROMOS C.xlsx", 'Octubre 2022')
DatosPolicromos_11_22 = adaptar_len_policromos("./altamira/2022/11. NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA POLICROMOS C.xlsx", 'Noviembre 2022')
DatosPolicromos_12_22 = adaptar_len_policromos("./altamira/2022/12. DICIEMBRE/ZONA CRUCE D.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA EXTERIOR.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA POLICROMOS.xlsx", 'Diciembre 2022')

# 0.2) Separar datos por semanas

### Para Cruce, exterior y hall:


In [4]:
def SepararPorSemanas(NombreFichero):
    df=pd.read_excel(NombreFichero)
    # convert the 'date' column to datetime format
    df['Fecha/Tiempo'] = pd.to_datetime(df['Fecha/Tiempo'])
    # extract the week number from the 'date' column
    df['week'] = df['Fecha/Tiempo'].dt.week
    # create an empty list to store the data of each week
    weekly_data = []
    # group the dataframe by 'week' starting on Monday
    grouped_df = df.groupby(pd.Grouper(key='Fecha/Tiempo', freq='W-SUN'))
    # loop through the grouped dataframe and append the data of each week to the list
    for week, data in grouped_df:
        weekly_data.append(data.reset_index(drop=True))
    return weekly_data 

In [7]:
tiempoInicial = time.time()
#Obtener los datos por semanas cruce:
#2021
SemanaCruceJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx")
SemanaCruceAgosto21 = SepararPorSemanas("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre21 = SepararPorSemanas("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")
SemanaCruceOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx")
#2022
SemanaCruceJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx")
SemanaCruceAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")

#Obtener los datos por semanas exterior:
#2021
SemanaExteriorJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto21 = SepararPorSemanas( "./altamira/2021/8.AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre21 = SepararPorSemanas( "./altamira/2021/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")
SemanaExteriorOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA EXTERIOR.xlsx")
#2022
SemanaExteriorJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")

#Obtener los datos por semanas hall:
#2021
SemanaHallJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA ENTRADA A.xlsx")
SemanaHallAgosto21 = SepararPorSemanas("./altamira/2021/8.AGOSTO/ZONA ENTRADA A.xlsx")
SemanaHallSeptiembre21 = SepararPorSemanas("./altamira/2021/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx")
SemanaHallOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA ENTRADA A.xlsx")
#2022
SemanaHallJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx")
SemanaHallAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA ENTRADA A.xlsx")
SemanaHallSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx")
tiempoFinal = time.time()
print(tiempoFinal-tiempoInicial)

16.847368240356445


In [ ]:
# Comprobar la longitud y si los datos por semanas empiezan y terminan a la misma hora:
def comprobarlenSemanas(SemanasCruce, SemanasExterior, SemanasHall, mesAño):
    print('--------------------------------------------------')
    print('--------------------------------------------------')
    print(mesAño)
    print(len(SemanasCruce), len(SemanasExterior), len(SemanasHall))
    if len(SemanasCruce)!=len(SemanasExterior) or len(SemanasCruce)!=len(SemanasHall) or len(SemanasExterior)!=len(SemanasHall):
        print('Hay más semanas en unos archivos que en otros')
    i=0
    while i<len(SemanasHall):
        print('--------------------------------------------------')
        print('i', 'len(Cruce)', 'len(Exterior)', 'len(Hall)')
        print(i, len(SemanasCruce[i]), len(SemanasExterior[i]), len(SemanasHall[i]))
        print('1er dato Cruce', '1er dato Exterior', '1er dato hall')
        print(SemanasCruce[i]['Fecha/Tiempo'][0], SemanasExterior[i]['Fecha/Tiempo'][0], SemanasHall[i]['Fecha/Tiempo'][0])
        print('ult dato Cruce', 'ult dato Exterior', 'ult dato hall')
        print(SemanasCruce[i]['Fecha/Tiempo'][len(SemanasCruce[i]['Fecha/Tiempo'])-1], 
              SemanasExterior[i]['Fecha/Tiempo'][len(SemanasExterior[i]['Fecha/Tiempo'])-1], 
              SemanasHall[i]['Fecha/Tiempo'][len(SemanasHall[i]['Fecha/Tiempo'])-1])  
        i=i+1

In [ ]:
# comprobarlenSemanas(SemanaCruceJulio21, SemanaExteriorJulio21, SemanaHallJulio21, 'Julio21')
# comprobarlenSemanas(SemanaCruceAgosto21, SemanaExteriorAgosto21, SemanaHallAgosto21, 'Agosto21')
# comprobarlenSemanas(SemanaCruceSeptiembre21, SemanaExteriorSeptiembre21, SemanaHallSeptiembre21, 'Septiembre21')
# comprobarlenSemanas(SemanaCruceOctubre21, SemanaExteriorOctubre21, SemanaHallOctubre21, 'Octubre21')
# comprobarlenSemanas(SemanaCruceJulio22, SemanaExteriorJulio22, SemanaHallJulio22, 'Julio22')
# comprobarlenSemanas(SemanaCruceAgosto22, SemanaExteriorAgosto22, SemanaHallAgosto22, 'Agosto22')
# comprobarlenSemanas(SemanaCruceSeptiembre22, SemanaExteriorSeptiembre22, SemanaHallSeptiembre22, 'Septiembre22')
# # Todo bn

### Separar por semanas policromos:

In [8]:
def separar_policromos_semanas(datosPolicromos):
    # datosPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo; Formato
    
    #Pasar lista de listas a diccionario:
    diccionario_policromos_10_20={'Tp': datosPolicromos[0], 'Tp_suavizada': datosPolicromos[1], 'CO2': datosPolicromos[2],
                                  'CO2_suavizado': datosPolicromos[3], 'Fecha/Tiempo': datosPolicromos[4]}
    #Pasar el diccionario a dataframe:
    df = pd.DataFrame(data=diccionario_policromos_10_20)
    #Separar por semanas:
    
    # convert the 'date' column to datetime format
    df['Fecha/Tiempo'] = pd.to_datetime(df['Fecha/Tiempo'])
    # extract the week number from the 'date' column
    df['week'] = df['Fecha/Tiempo'].dt.week
    # create an empty list to store the data of each week
    weekly_data = []
    # group the dataframe by 'week' starting on Monday
    grouped_df = df.groupby(pd.Grouper(key='Fecha/Tiempo', freq='W-SUN'))
    # loop through the grouped dataframe and append the data of each week to the list
    for week, data in grouped_df:
        weekly_data.append(data.reset_index(drop=True))
    return weekly_data 

In [ ]:
#Separar por semanas los meses con oscilación clara:
#2021
PolicromosSemanas_7_2021 = separar_policromos_semanas(DatosPolicromos_7_21)
PolicromosSemanas_8_2021 = separar_policromos_semanas(DatosPolicromos_8_21)
PolicromosSemanas_9_2021 = separar_policromos_semanas(DatosPolicromos_9_21)
PolicromosSemanas_10_2021 = separar_policromos_semanas(DatosPolicromos_10_21)
#2022
PolicromosSemanas_7_2022 = separar_policromos_semanas(DatosPolicromos_7_22)     
PolicromosSemanas_8_2022 = separar_policromos_semanas(DatosPolicromos_8_22)
PolicromosSemanas_9_2022 = separar_policromos_semanas(DatosPolicromos_9_22)


In [12]:
# No hacen falta todos los meses por semanas, solo los que tienen oscilaciones claras:
# 2020
# PolicromosSemanas_10_2020 = separar_policromos_semanas(DatosPolicromos_10_20)     
# PolicromosSemanas_11_2020 = separar_policromos_semanas(DatosPolicromos_11_20)
# PolicromosSemanas_12_2020 = separar_policromos_semanas(DatosPolicromos_12_20)
# #2021
# PolicromosSemanas_1_2021 = separar_policromos_semanas(DatosPolicromos_1_21)
# PolicromosSemanas_2_2021 = separar_policromos_semanas(DatosPolicromos_2_21)
# PolicromosSemanas_3_2021 = separar_policromos_semanas(DatosPolicromos_3_21)
# PolicromosSemanas_4_2021 = separar_policromos_semanas(DatosPolicromos_4_21)
# PolicromosSemanas_5_2021 = separar_policromos_semanas(DatosPolicromos_5_21)
# PolicromosSemanas_6_2021 = separar_policromos_semanas(DatosPolicromos_6_21)
# PolicromosSemanas_7_2021 = separar_policromos_semanas(DatosPolicromos_7_21)
# PolicromosSemanas_8_2021 = separar_policromos_semanas(DatosPolicromos_8_21)
# PolicromosSemanas_9_2021 = separar_policromos_semanas(DatosPolicromos_9_21)
# PolicromosSemanas_10_2021 = separar_policromos_semanas(DatosPolicromos_10_21)
# PolicromosSemanas_11_2021 = separar_policromos_semanas(DatosPolicromos_11_21)
# PolicromosSemanas_12_2021 = separar_policromos_semanas(DatosPolicromos_12_21)
# 2022
# PolicromosSemanas_1_2022 = separar_policromos_semanas(DatosPolicromos_1_22)
# PolicromosSemanas_2_2022 = separar_policromos_semanas(DatosPolicromos_2_22)
# PolicromosSemanas_3_2022 = separar_policromos_semanas(DatosPolicromos_3_22)
# PolicromosSemanas_4_2022 = separar_policromos_semanas(DatosPolicromos_4_22)
# PolicromosSemanas_5_2022 = separar_policromos_semanas(DatosPolicromos_5_22)
# PolicromosSemanas_6_2022 = separar_policromos_semanas(DatosPolicromos_6_22)
# PolicromosSemanas_7_2022 = separar_policromos_semanas(DatosPolicromos_7_22)     
# PolicromosSemanas_8_2022 = separar_policromos_semanas(DatosPolicromos_8_22)
# PolicromosSemanas_9_2022 = separar_policromos_semanas(DatosPolicromos_9_22)
# PolicromosSemanas_10_2022 = separar_policromos_semanas(DatosPolicromos_10_22)
# PolicromosSemanas_11_2022 = separar_policromos_semanas(DatosPolicromos_11_22)
# PolicromosSemanas_12_2022 = separar_policromos_semanas(DatosPolicromos_12_22)

,Tp,Tp_suavizada,CO2,CO2_suavizado,Fecha/Tiempo,week
0,14.77,14.772,2111,2106.4,2020-10-01 00:15:55.090,40
1,14.77,14.780,2094,2105.4,2020-10-01 00:30:55.090,40
2,14.77,14.770,2108,2107.8,2020-10-01 00:45:55.090,40
3,14.76,14.760,2094,2109.0,2020-10-01 01:15:55.090,40
4,14.76,14.764,2109,2112.8,2020-10-01 01:30:55.090,40
...,...,...,...,...,...,...
283,14.85,14.844,3754,3753.6,2020-10-04 22:30:55.090,40
284,14.85,14.848,3767,3735.6,2020-10-04 22:45:55.090,40
285,14.84,14.844,3738,3729.4,2020-10-04 23:15:55.090,40
286,14.85,14.846,3737,3731.2,2020-10-04 23:30:55.090,40


¿Los dataframes de Cruce, Exterior, Hall y Policromos por semanas tiene todos el mismo numero de semanas?
Hacer si da errores

# 1) ¿Las oscilaciones diarias se deben a un intercambio de aire con el exterior? Coeficientes de correlación entre la diferencia de temperaturas entre exterior y cada sala con la cantidad de CO2. Variables sin suavizar

Tengo: Listas de dataframes que contienen en cada elemento los datos de una semana del mes

In [13]:
def difT_CO2_cc_por_semanas(DataframeCruce, DataframeExterior, DataframeHall, PolicromosSemanas):
    """
    DataframeCruce, DataframeExterior, DataframeHall son todos listas de dataframes para cada semana, al igual que 
    PolicromosSemanas.
    Columnas de PolicromosSemanas= Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo
    """
    i=0
    r=[]
    while i<len(DataframeCruce):
       #print(i)
        DataframeCruce[i].reset_index(drop=True, inplace=True)
        DataframeExterior[i].reset_index(drop=True, inplace=True)
        DataframeHall[i].reset_index(drop=True, inplace=True)
        PolicromosSemanas[i].reset_index(drop=True, inplace=True)
        
        # Variables que me van a hacer falta: Tc, Te, Th, Tp, CO2c, CO2h, CO2p
        for index, _ in DataframeExterior[i].iterrows():#Se va fuera de indice
            if  __builtins__.sum([pd.isna(DataframeCruce[i]["Temperatura D (° C)"][index]),
                                  pd.isna(DataframeCruce[i]['CO2 D (ppm)'][index]),
                                  pd.isna(DataframeHall[i]['Temperatura A (° C)'][index]),
                                  pd.isna(DataframeHall[i]['CO2 A (ppm)'][index]),
                                  pd.isna(PolicromosSemanas[i]['Tp'][index]),
                                  pd.isna(PolicromosSemanas[i]["CO2"][index]),
                                  pd.isna(DataframeExterior[i]["Tr Exterior (C Tr)"][index])]) > 0:
                DataframeCruce[i].drop([index], axis=0, inplace=True)
                DataframeExterior[i].drop([index], axis=0, inplace=True) 
                DataframeHall[i].drop([index], axis=0, inplace=True) 
                PolicromosSemanas[i].drop([index], axis=0, inplace=True) 
                
        #Quiero los coef entre CO2 y dif de temperaturas:
        Dif_T_cruce= DataframeExterior[i]["Tr Exterior (C Tr)"]-DataframeCruce[i]["Temperatura D (° C)"]
        Dif_T_Hall = DataframeExterior[i]["Tr Exterior (C Tr)"]-DataframeHall[i]['Temperatura A (° C)']
        Dif_T_Policromos = DataframeExterior[i]["Tr Exterior (C Tr)"]-PolicromosSemanas[i]['Tp']
        #Mediante scipy:
        r_difT_CO2_cruce, p_difT_CO2_cruce = stats.pearsonr(Dif_T_cruce, DataframeCruce[i]["CO2 D (ppm)"])
        r_difT_CO2_hall, p_difT_CO2_hall = stats.pearsonr(Dif_T_Hall, DataframeHall[i]["CO2 A (ppm)"])
        r_difT_CO2_policromos, p_difT_CO2_policromos = stats.pearsonr(Dif_T_Policromos, PolicromosSemanas[i]["CO2"])
        rs = r_difT_CO2_cruce, p_difT_CO2_cruce, r_difT_CO2_hall, p_difT_CO2_hall, r_difT_CO2_policromos, p_difT_CO2_policromos
        r.append(rs)
        i=i+1
    return(r)

In [22]:
#Separar por semanas los meses con oscilación clara:
#2021
r7_2021 = difT_CO2_cc_por_semanas(SemanaCruceJulio21, SemanaExteriorJulio21, SemanaHallJulio21, PolicromosSemanas_7_2021)
r8_2021 = difT_CO2_cc_por_semanas(SemanaCruceAgosto21, SemanaExteriorAgosto21, SemanaHallAgosto21, PolicromosSemanas_8_2021)
r9_2021 = difT_CO2_cc_por_semanas(SemanaCruceSeptiembre21, SemanaExteriorSeptiembre21, SemanaHallSeptiembre21, 
                                  PolicromosSemanas_9_2021)
r10_2021 = difT_CO2_cc_por_semanas(SemanaCruceOctubre21, SemanaExteriorOctubre21, SemanaHallOctubre21, 
                                   PolicromosSemanas_10_2021)
#2022
r7_2022 = difT_CO2_cc_por_semanas(SemanaCruceJulio22, SemanaExteriorJulio22, SemanaHallJulio22, PolicromosSemanas_7_2022)     
r8_2022 = difT_CO2_cc_por_semanas(SemanaCruceAgosto22, SemanaExteriorAgosto22, SemanaHallAgosto22, PolicromosSemanas_8_2022)
r9_2022 = difT_CO2_cc_por_semanas_semanas(SemanaCruceSeptiembre22, SemanaExteriorSeptiembre22, SemanaHallSeptiembre22, 
                                     PolicromosSemanas_9_2022)

In [32]:
s1=pd.Series([r7_2021[0][0],r7_2021[1][0], r7_2021[2][0],r7_2021[3][0],r7_2021[4][0],
              r8_2021[0][0],r8_2021[1][0], r8_2021[2][0],r8_2021[3][0],r8_2021[4][0],#semanas de agosto 21
              r9_2021[0][0],r9_2021[1][0],r9_2021[2][0],r9_2021[3][0],r9_2021[4][0],#semanas de septiembre 21
              r10_2021[0][0],r10_2021[1][0],r10_2021[2][0],r10_2021[3][0],r10_2021[4][0],
              r7_2022[0][0], r7_2022[1][0], r7_2022[2][0], r7_2022[3][0], r7_2022[4][0],  
              r8_2022[0][0], r8_2022[1][0], r8_2022[2][0], r8_2022[3][0], r8_2022[4][0],
              r9_2022[0][0], r9_2022[1][0], r9_2022[2][0], r9_2022[3][0], r9_2022[4][0]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s2=pd.Series([r7_2021[0][1],r7_2021[1][1], r7_2021[2][1],r7_2021[3][1],r7_2021[4][1],
              r8_2021[0][1],r8_2021[1][1], r8_2021[2][1],r8_2021[3][1],r8_2021[4][1],#semanas de agosto 21
              r9_2021[0][1],r9_2021[1][1],r9_2021[2][1],r9_2021[3][1],r9_2021[4][1],#semanas de septiembre 21
              r10_2021[0][1],r10_2021[1][1],r10_2021[2][1],r10_2021[3][1],r10_2021[4][1],
              r7_2022[0][1], r7_2022[1][1], r7_2022[2][1], r7_2022[3][1], r7_2022[4][1],  
              r8_2022[0][1], r8_2022[1][1], r8_2022[2][1], r8_2022[3][1], r8_2022[4][1],
              r9_2022[0][1], r9_2022[1][1], r9_2022[2][1], r9_2022[3][1], r9_2022[4][1]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s3=pd.Series([r7_2021[0][2],r7_2021[1][2], r7_2021[2][2],r7_2021[3][2],r7_2021[4][2],
              r8_2021[0][2],r8_2021[1][2], r8_2021[2][2],r8_2021[3][2],r8_2021[4][2],#semanas de agosto 21
              r9_2021[0][2],r9_2021[1][2],r9_2021[2][2],r9_2021[3][2],r9_2021[4][2],#semanas de septiembre 21
              r10_2021[0][2],r10_2021[1][2],r10_2021[2][2],r10_2021[3][2],r10_2021[4][2],
              r7_2022[0][2], r7_2022[1][2], r7_2022[2][2], r7_2022[3][2], r7_2022[4][2], 
              r8_2022[0][2], r8_2022[1][2], r8_2022[2][2], r8_2022[3][2], r8_2022[4][2],
              r9_2022[0][2], r9_2022[1][2], r9_2022[2][2], r9_2022[3][2], r9_2022[4][2]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s4=pd.Series([r7_2021[0][3],r7_2021[1][3], r7_2021[2][3],r7_2021[3][3],r7_2021[4][3],
              r8_2021[0][3],r8_2021[1][3], r8_2021[2][3],r8_2021[3][3],r8_2021[4][3],#semanas de agosto 21
              r9_2021[0][3],r9_2021[1][3],r9_2021[2][3],r9_2021[3][3],r9_2021[4][3],
              r10_2021[0][3],r10_2021[1][3], r10_2021[2][3],r10_2021[3][3],r10_2021[4][3],
              r7_2022[0][3], r7_2022[1][3], r7_2022[2][3], r7_2022[3][3], r7_2022[4][3],
              r8_2022[0][3], r8_2022[1][3], r8_2022[2][3], r8_2022[3][3], r8_2022[4][3],
              r9_2022[0][3], r9_2022[1][3], r9_2022[2][3], r9_2022[3][3], r9_2022[4][3]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s5=pd.Series([r7_2021[0][4],r7_2021[1][4], r7_2021[2][4],r7_2021[3][4],r7_2021[4][4],
              r8_2021[0][4],r8_2021[1][4], r8_2021[2][4],r8_2021[3][4],r8_2021[4][4],#semanas de agosto 21
              r9_2021[0][4],r9_2021[1][4],r9_2021[2][4],r9_2021[3][4],r9_2021[4][4],#semanas de septiembre 21
              r10_2021[0][4],r10_2021[1][4],r10_2021[2][4],r10_2021[3][4],r10_2021[4][4],
              r7_2022[0][4], r7_2022[1][4], r7_2022[2][4], r7_2022[3][4], r7_2022[4][4], 
              r8_2022[0][4], r8_2022[1][4], r8_2022[2][4], r8_2022[3][4], r8_2022[4][4],
              r9_2022[0][4], r9_2022[1][4], r9_2022[2][4], r9_2022[3][4], r9_2022[4][4]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s6=pd.Series([r7_2021[0][5],r7_2021[1][5], r7_2021[2][5],r7_2021[3][5],r7_2021[4][5],
              r8_2021[0][5],r8_2021[1][5], r8_2021[2][5],r8_2021[3][5],r8_2021[4][5],#semanas de agosto 21
              r9_2021[0][5],r9_2021[1][5],r9_2021[2][5],r9_2021[3][5],r9_2021[4][5],#semanas de septiembre 21
              r10_2021[0][5],r10_2021[1][5],r10_2021[2][5],r10_2021[3][5],r10_2021[4][5],
              r7_2022[0][5], r7_2022[1][5], r7_2022[2][5], r7_2022[3][5], r7_2022[4][5],
              r8_2022[0][5], r8_2022[1][5], r8_2022[2][5], r8_2022[3][5], r8_2022[4][5],
              r9_2022[0][5], r9_2022[1][5], r9_2022[2][5], r9_2022[3][5], r9_2022[4][5]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])

#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['CC_difT_CO2_cruce', 'pvalue_difT_CO2_cruce', 'CC_difT_CO2_hall',  'pvalue_hall', 
                      'CC_difT_CO2_policromos',  'pvalue_polimos']
display(CoefCorrel)

,CC_difT_CO2_cruce,pvalue_difT_CO2_cruce,CC_difT_CO2_hall,pvalue_hall,CC_difT_CO2_policromos,pvalue_polimos
Sem1 Julio 2021,-0.666723,1.039730e-50,-0.386983,3.626907e-15,-0.382689,7.693070e-15
Sem2 Julio 2021,-0.670508,8.392440e-89,-0.461425,1.102193e-36,-0.106301,5.846332e-03
Sem3 Julio 2021,-0.647755,3.376392e-81,-0.326137,4.061759e-18,-0.009628,8.032563e-01
Sem4 Julio 2021,-0.550807,1.406587e-54,-0.125804,1.082580e-03,0.332825,7.599153e-19
Sem5 Julio 2021,-0.698186,2.335825e-85,-0.508998,2.741988e-39,-0.244821,2.620378e-09
Sem1 Agosto 2021,-0.809142,1.988152e-23,-0.519383,5.890440e-08,0.214521,3.583188e-02
Sem2 Agosto 2021,-0.635469,2.728410e-77,-0.278907,1.794923e-13,0.132164,5.930139e-04
Sem3 Agosto 2021,-0.693486,2.467955e-97,-0.158082,3.902851e-05,-0.160570,2.931300e-05
Sem4 Agosto 2021,-0.603726,5.675587e-68,-0.132187,5.916957e-04,0.227621,2.399872e-09
Sem5 Agosto 2021,-0.569650,1.304487e-29,-0.212867,1.022928e-04,0.351847,5.424144e-11


# 1.2) ¿Las oscilaciones diarias se deben a un intercambio de aire con el exterior? Coeficientes de correlación entre la diferencia de temperaturas entre exterior y cada sala con la cantidad de CO2. Variables suavizadas

In [35]:
def difT_CO2_cc_por_semanas(DataframeCruce, DataframeExterior, DataframeHall, PolicromosSemanas):
    """
    DataframeCruce, DataframeExterior, DataframeHall son todos listas de dataframes para cada semana, al igual que 
    PolicromosSemanas.
    Columnas de PolicromosSemanas= Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo
    """
    i=0
    r=[]
    while i<len(DataframeCruce):
       #print(i)
        DataframeCruce[i].reset_index(drop=True, inplace=True)
        DataframeExterior[i].reset_index(drop=True, inplace=True)
        DataframeHall[i].reset_index(drop=True, inplace=True)
        PolicromosSemanas[i].reset_index(drop=True, inplace=True)
        
        # Variables que me van a hacer falta: Tc, Te, Th, Tp, CO2c, CO2h, CO2p
        for index, _ in DataframeExterior[i].iterrows():#Se va fuera de indice
            if  __builtins__.sum([pd.isna(DataframeCruce[i]['Tc_suavizada'][index]),
                                  pd.isna(DataframeCruce[i]['CO2c_suavizado'][index]),
                                  pd.isna(DataframeHall[i]['Th_suavizada'][index]),
                                  pd.isna(DataframeHall[i]['CO2h_suavizado'][index]),
                                  pd.isna(PolicromosSemanas[i]['Tp_suavizada'][index]),
                                  pd.isna(PolicromosSemanas[i]["CO2_suavizado"][index]),
                                  pd.isna(DataframeExterior[i]['Te_suavizada'][index])]) > 0:
                DataframeCruce[i].drop([index], axis=0, inplace=True)
                DataframeExterior[i].drop([index], axis=0, inplace=True) 
                DataframeHall[i].drop([index], axis=0, inplace=True) 
                PolicromosSemanas[i].drop([index], axis=0, inplace=True) 
                
        #Quiero los coef entre CO2 y dif de temperaturas:
        Dif_T_cruce= DataframeExterior[i]['Te_suavizada']-DataframeCruce[i]['Tc_suavizada']
        Dif_T_Hall = DataframeExterior[i]['Te_suavizada']-DataframeHall[i]['Th_suavizada']
        Dif_T_Policromos = DataframeExterior[i]['Te_suavizada']-PolicromosSemanas[i]['Tp_suavizada']
        #Mediante scipy:
        r_difT_CO2_cruce, p_difT_CO2_cruce = stats.pearsonr(Dif_T_cruce, DataframeCruce[i]['CO2c_suavizado'])
        r_difT_CO2_hall, p_difT_CO2_hall = stats.pearsonr(Dif_T_Hall, DataframeHall[i]['CO2h_suavizado'])
        r_difT_CO2_policromos, p_difT_CO2_policromos = stats.pearsonr(Dif_T_Policromos, PolicromosSemanas[i]['CO2_suavizado'])
        rs = r_difT_CO2_cruce, p_difT_CO2_cruce, r_difT_CO2_hall, p_difT_CO2_hall, r_difT_CO2_policromos, p_difT_CO2_policromos
        r.append(rs)
        i=i+1
    return(r)

In [37]:
#Separar por semanas los meses con oscilación clara:
#2021
r7_2021 = difT_CO2_cc_por_semanas(SemanaCruceJulio21, SemanaExteriorJulio21, SemanaHallJulio21, PolicromosSemanas_7_2021)
r8_2021 = difT_CO2_cc_por_semanas(SemanaCruceAgosto21, SemanaExteriorAgosto21, SemanaHallAgosto21, PolicromosSemanas_8_2021)
r9_2021 = difT_CO2_cc_por_semanas(SemanaCruceSeptiembre21, SemanaExteriorSeptiembre21, SemanaHallSeptiembre21, 
                                  PolicromosSemanas_9_2021)
r10_2021 = difT_CO2_cc_por_semanas(SemanaCruceOctubre21, SemanaExteriorOctubre21, SemanaHallOctubre21, 
                                   PolicromosSemanas_10_2021)
#2022
r7_2022 = difT_CO2_cc_por_semanas(SemanaCruceJulio22, SemanaExteriorJulio22, SemanaHallJulio22, PolicromosSemanas_7_2022)     
r8_2022 = difT_CO2_cc_por_semanas(SemanaCruceAgosto22, SemanaExteriorAgosto22, SemanaHallAgosto22, PolicromosSemanas_8_2022)
r9_2022 = difT_CO2_cc_por_semanas(SemanaCruceSeptiembre22, SemanaExteriorSeptiembre22, SemanaHallSeptiembre22, 
                                     PolicromosSemanas_9_2022)

In [38]:
s1=pd.Series([r7_2021[0][0],r7_2021[1][0], r7_2021[2][0],r7_2021[3][0],r7_2021[4][0],
              r8_2021[0][0],r8_2021[1][0], r8_2021[2][0],r8_2021[3][0],r8_2021[4][0],#semanas de agosto 21
              r9_2021[0][0],r9_2021[1][0],r9_2021[2][0],r9_2021[3][0],r9_2021[4][0],#semanas de septiembre 21
              r10_2021[0][0],r10_2021[1][0],r10_2021[2][0],r10_2021[3][0],r10_2021[4][0],
              r7_2022[0][0], r7_2022[1][0], r7_2022[2][0], r7_2022[3][0], r7_2022[4][0],  
              r8_2022[0][0], r8_2022[1][0], r8_2022[2][0], r8_2022[3][0], r8_2022[4][0],
              r9_2022[0][0], r9_2022[1][0], r9_2022[2][0], r9_2022[3][0], r9_2022[4][0]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s2=pd.Series([r7_2021[0][1],r7_2021[1][1], r7_2021[2][1],r7_2021[3][1],r7_2021[4][1],
              r8_2021[0][1],r8_2021[1][1], r8_2021[2][1],r8_2021[3][1],r8_2021[4][1],#semanas de agosto 21
              r9_2021[0][1],r9_2021[1][1],r9_2021[2][1],r9_2021[3][1],r9_2021[4][1],#semanas de septiembre 21
              r10_2021[0][1],r10_2021[1][1],r10_2021[2][1],r10_2021[3][1],r10_2021[4][1],
              r7_2022[0][1], r7_2022[1][1], r7_2022[2][1], r7_2022[3][1], r7_2022[4][1],  
              r8_2022[0][1], r8_2022[1][1], r8_2022[2][1], r8_2022[3][1], r8_2022[4][1],
              r9_2022[0][1], r9_2022[1][1], r9_2022[2][1], r9_2022[3][1], r9_2022[4][1]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s3=pd.Series([r7_2021[0][2],r7_2021[1][2], r7_2021[2][2],r7_2021[3][2],r7_2021[4][2],
              r8_2021[0][2],r8_2021[1][2], r8_2021[2][2],r8_2021[3][2],r8_2021[4][2],#semanas de agosto 21
              r9_2021[0][2],r9_2021[1][2],r9_2021[2][2],r9_2021[3][2],r9_2021[4][2],#semanas de septiembre 21
              r10_2021[0][2],r10_2021[1][2],r10_2021[2][2],r10_2021[3][2],r10_2021[4][2],
              r7_2022[0][2], r7_2022[1][2], r7_2022[2][2], r7_2022[3][2], r7_2022[4][2], 
              r8_2022[0][2], r8_2022[1][2], r8_2022[2][2], r8_2022[3][2], r8_2022[4][2],
              r9_2022[0][2], r9_2022[1][2], r9_2022[2][2], r9_2022[3][2], r9_2022[4][2]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s4=pd.Series([r7_2021[0][3],r7_2021[1][3], r7_2021[2][3],r7_2021[3][3],r7_2021[4][3],
              r8_2021[0][3],r8_2021[1][3], r8_2021[2][3],r8_2021[3][3],r8_2021[4][3],#semanas de agosto 21
              r9_2021[0][3],r9_2021[1][3],r9_2021[2][3],r9_2021[3][3],r9_2021[4][3],
              r10_2021[0][3],r10_2021[1][3], r10_2021[2][3],r10_2021[3][3],r10_2021[4][3],
              r7_2022[0][3], r7_2022[1][3], r7_2022[2][3], r7_2022[3][3], r7_2022[4][3],
              r8_2022[0][3], r8_2022[1][3], r8_2022[2][3], r8_2022[3][3], r8_2022[4][3],
              r9_2022[0][3], r9_2022[1][3], r9_2022[2][3], r9_2022[3][3], r9_2022[4][3]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s5=pd.Series([r7_2021[0][4],r7_2021[1][4], r7_2021[2][4],r7_2021[3][4],r7_2021[4][4],
              r8_2021[0][4],r8_2021[1][4], r8_2021[2][4],r8_2021[3][4],r8_2021[4][4],#semanas de agosto 21
              r9_2021[0][4],r9_2021[1][4],r9_2021[2][4],r9_2021[3][4],r9_2021[4][4],#semanas de septiembre 21
              r10_2021[0][4],r10_2021[1][4],r10_2021[2][4],r10_2021[3][4],r10_2021[4][4],
              r7_2022[0][4], r7_2022[1][4], r7_2022[2][4], r7_2022[3][4], r7_2022[4][4], 
              r8_2022[0][4], r8_2022[1][4], r8_2022[2][4], r8_2022[3][4], r8_2022[4][4],
              r9_2022[0][4], r9_2022[1][4], r9_2022[2][4], r9_2022[3][4], r9_2022[4][4]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s6=pd.Series([r7_2021[0][5],r7_2021[1][5], r7_2021[2][5],r7_2021[3][5],r7_2021[4][5],
              r8_2021[0][5],r8_2021[1][5], r8_2021[2][5],r8_2021[3][5],r8_2021[4][5],#semanas de agosto 21
              r9_2021[0][5],r9_2021[1][5],r9_2021[2][5],r9_2021[3][5],r9_2021[4][5],#semanas de septiembre 21
              r10_2021[0][5],r10_2021[1][5],r10_2021[2][5],r10_2021[3][5],r10_2021[4][5],
              r7_2022[0][5], r7_2022[1][5], r7_2022[2][5], r7_2022[3][5], r7_2022[4][5],
              r8_2022[0][5], r8_2022[1][5], r8_2022[2][5], r8_2022[3][5], r8_2022[4][5],
              r9_2022[0][5], r9_2022[1][5], r9_2022[2][5], r9_2022[3][5], r9_2022[4][5]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])

#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['CC_difT_CO2_cruce', 'pvalue_difT_CO2_cruce', 'CC_difT_CO2_hall',  'pvalue_hall', 
                      'CC_difT_CO2_policromos',  'pvalue_polimos']
display(CoefCorrel)

,CC_difT_CO2_cruce,pvalue_difT_CO2_cruce,CC_difT_CO2_hall,pvalue_hall,CC_difT_CO2_policromos,pvalue_polimos
Sem1 Julio 2021,-0.683940,1.400426e-53,-0.395945,1.123222e-15,-0.409520,9.285915e-17
Sem2 Julio 2021,-0.674210,1.825992e-89,-0.466596,2.613888e-37,-0.149039,1.131117e-04
Sem3 Julio 2021,-0.651230,2.456403e-82,-0.327164,3.148010e-18,-0.056011,1.469421e-01
Sem4 Julio 2021,-0.558351,2.422426e-56,-0.136756,3.776415e-04,0.274725,4.218461e-13
Sem5 Julio 2021,-0.701407,1.841988e-86,-0.508344,3.551225e-39,-0.277360,1.239222e-11
Sem1 Agosto 2021,-0.825118,8.648015e-24,-0.549436,1.700458e-08,0.035070,7.413810e-01
Sem2 Agosto 2021,-0.642507,1.658126e-79,-0.287075,3.241997e-14,0.078528,4.184730e-02
Sem3 Agosto 2021,-0.706378,9.904575e-102,-0.167034,1.470824e-05,-0.237996,4.971302e-10
Sem4 Agosto 2021,-0.605524,1.794748e-68,-0.136353,3.931074e-04,0.158890,3.510376e-05
Sem5 Agosto 2021,-0.562379,4.406917e-28,-0.247821,7.257993e-06,0.303705,2.972553e-08


# 2) ¿Hay relación entre la cantidad de CO2 que hay en cada sala? Coef correl CO2c-CO2h y CO2c-CO2p con las variables sin suavizar

In [48]:
def CO2_entreSalas_cc_por_semanas(DataframeCruce, DataframeHall, PolicromosSemanas):
    """
    DataframeCruce, DataframeExterior, DataframeHall son todos listas de dataframes para cada semana, al igual que 
    PolicromosSemanas.
    Columnas de PolicromosSemanas= Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo
    """
    i=0
    r=[]
    while i<len(DataframeCruce):
       #print(i)
        DataframeCruce[i].reset_index(drop=True, inplace=True)
        DataframeHall[i].reset_index(drop=True, inplace=True)
        PolicromosSemanas[i].reset_index(drop=True, inplace=True)
        
        # Variables que me van a hacer falta: Tc, Te, Th, Tp, CO2c, CO2h, CO2p
        for index, _ in DataframeHall[i].iterrows():#Se va fuera de indice
            if  __builtins__.sum([pd.isna(DataframeCruce[i]['CO2 D (ppm)'][index]),
                                  pd.isna(DataframeHall[i]['CO2 A (ppm)'][index]),
                                  pd.isna(PolicromosSemanas[i]["CO2"][index])]) > 0:
                DataframeCruce[i].drop([index], axis=0, inplace=True)
                DataframeHall[i].drop([index], axis=0, inplace=True) 
                PolicromosSemanas[i].drop([index], axis=0, inplace=True) 
        #Mediante scipy:
        CO2cc_ch, CO2p_ch = stats.pearsonr(DataframeCruce[i]['CO2 D (ppm)'], DataframeHall[i]['CO2 A (ppm)'])
        CO2cc_cp, CO2p_cp = stats.pearsonr(DataframeCruce[i]['CO2 D (ppm)'], PolicromosSemanas[i]["CO2"])
        CO2cc_hp, CO2p_hp = stats.pearsonr(DataframeHall[i]['CO2 A (ppm)'], PolicromosSemanas[i]["CO2"])
        rs= CO2cc_ch, CO2p_ch, CO2cc_cp, CO2p_cp, CO2cc_hp, CO2p_hp
        r.append(rs)
        i=i+1
    return(r)

In [49]:
#Separar por semanas los meses con oscilación clara:
#2021
r7_2021 = CO2_entreSalas_cc_por_semanas(SemanaCruceJulio21, SemanaHallJulio21, PolicromosSemanas_7_2021)
r8_2021 = CO2_entreSalas_cc_por_semanas(SemanaCruceAgosto21, SemanaHallAgosto21, PolicromosSemanas_8_2021)
r9_2021 = CO2_entreSalas_cc_por_semanas(SemanaCruceSeptiembre21, SemanaHallSeptiembre21, PolicromosSemanas_9_2021)
r10_2021 = CO2_entreSalas_cc_por_semanas(SemanaCruceOctubre21, SemanaHallOctubre21, PolicromosSemanas_10_2021)
#2022
r7_2022 = CO2_entreSalas_cc_por_semanas(SemanaCruceJulio22, SemanaHallJulio22, PolicromosSemanas_7_2022)     
r8_2022 = CO2_entreSalas_cc_por_semanas(SemanaCruceAgosto22, SemanaHallAgosto22, PolicromosSemanas_8_2022)
r9_2022 = CO2_entreSalas_cc_por_semanas(SemanaCruceSeptiembre22, SemanaHallSeptiembre22, PolicromosSemanas_9_2022)

In [50]:
s1=pd.Series([r7_2021[0][0],r7_2021[1][0], r7_2021[2][0],r7_2021[3][0],r7_2021[4][0],
              r8_2021[0][0],r8_2021[1][0], r8_2021[2][0],r8_2021[3][0],r8_2021[4][0],#semanas de agosto 21
              r9_2021[0][0],r9_2021[1][0],r9_2021[2][0],r9_2021[3][0],r9_2021[4][0],#semanas de septiembre 21
              r10_2021[0][0],r10_2021[1][0],r10_2021[2][0],r10_2021[3][0],r10_2021[4][0],
              r7_2022[0][0], r7_2022[1][0], r7_2022[2][0], r7_2022[3][0], r7_2022[4][0],  
              r8_2022[0][0], r8_2022[1][0], r8_2022[2][0], r8_2022[3][0], r8_2022[4][0],
              r9_2022[0][0], r9_2022[1][0], r9_2022[2][0], r9_2022[3][0], r9_2022[4][0]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s2=pd.Series([r7_2021[0][1],r7_2021[1][1], r7_2021[2][1],r7_2021[3][1],r7_2021[4][1],
              r8_2021[0][1],r8_2021[1][1], r8_2021[2][1],r8_2021[3][1],r8_2021[4][1],#semanas de agosto 21
              r9_2021[0][1],r9_2021[1][1],r9_2021[2][1],r9_2021[3][1],r9_2021[4][1],#semanas de septiembre 21
              r10_2021[0][1],r10_2021[1][1],r10_2021[2][1],r10_2021[3][1],r10_2021[4][1],
              r7_2022[0][1], r7_2022[1][1], r7_2022[2][1], r7_2022[3][1], r7_2022[4][1],  
              r8_2022[0][1], r8_2022[1][1], r8_2022[2][1], r8_2022[3][1], r8_2022[4][1],
              r9_2022[0][1], r9_2022[1][1], r9_2022[2][1], r9_2022[3][1], r9_2022[4][1]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s3=pd.Series([r7_2021[0][2],r7_2021[1][2], r7_2021[2][2],r7_2021[3][2],r7_2021[4][2],
              r8_2021[0][2],r8_2021[1][2], r8_2021[2][2],r8_2021[3][2],r8_2021[4][2],#semanas de agosto 21
              r9_2021[0][2],r9_2021[1][2],r9_2021[2][2],r9_2021[3][2],r9_2021[4][2],#semanas de septiembre 21
              r10_2021[0][2],r10_2021[1][2],r10_2021[2][2],r10_2021[3][2],r10_2021[4][2],
              r7_2022[0][2], r7_2022[1][2], r7_2022[2][2], r7_2022[3][2], r7_2022[4][2], 
              r8_2022[0][2], r8_2022[1][2], r8_2022[2][2], r8_2022[3][2], r8_2022[4][2],
              r9_2022[0][2], r9_2022[1][2], r9_2022[2][2], r9_2022[3][2], r9_2022[4][2]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s4=pd.Series([r7_2021[0][3],r7_2021[1][3], r7_2021[2][3],r7_2021[3][3],r7_2021[4][3],
              r8_2021[0][3],r8_2021[1][3], r8_2021[2][3],r8_2021[3][3],r8_2021[4][3],#semanas de agosto 21
              r9_2021[0][3],r9_2021[1][3],r9_2021[2][3],r9_2021[3][3],r9_2021[4][3],
              r10_2021[0][3],r10_2021[1][3], r10_2021[2][3],r10_2021[3][3],r10_2021[4][3],
              r7_2022[0][3], r7_2022[1][3], r7_2022[2][3], r7_2022[3][3], r7_2022[4][3],
              r8_2022[0][3], r8_2022[1][3], r8_2022[2][3], r8_2022[3][3], r8_2022[4][3],
              r9_2022[0][3], r9_2022[1][3], r9_2022[2][3], r9_2022[3][3], r9_2022[4][3]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s5=pd.Series([r7_2021[0][4],r7_2021[1][4], r7_2021[2][4],r7_2021[3][4],r7_2021[4][4],
              r8_2021[0][4],r8_2021[1][4], r8_2021[2][4],r8_2021[3][4],r8_2021[4][4],#semanas de agosto 21
              r9_2021[0][4],r9_2021[1][4],r9_2021[2][4],r9_2021[3][4],r9_2021[4][4],#semanas de septiembre 21
              r10_2021[0][4],r10_2021[1][4],r10_2021[2][4],r10_2021[3][4],r10_2021[4][4],
              r7_2022[0][4], r7_2022[1][4], r7_2022[2][4], r7_2022[3][4], r7_2022[4][4], 
              r8_2022[0][4], r8_2022[1][4], r8_2022[2][4], r8_2022[3][4], r8_2022[4][4],
              r9_2022[0][4], r9_2022[1][4], r9_2022[2][4], r9_2022[3][4], r9_2022[4][4]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s6=pd.Series([r7_2021[0][5],r7_2021[1][5], r7_2021[2][5],r7_2021[3][5],r7_2021[4][5],
              r8_2021[0][5],r8_2021[1][5], r8_2021[2][5],r8_2021[3][5],r8_2021[4][5],#semanas de agosto 21
              r9_2021[0][5],r9_2021[1][5],r9_2021[2][5],r9_2021[3][5],r9_2021[4][5],#semanas de septiembre 21
              r10_2021[0][5],r10_2021[1][5],r10_2021[2][5],r10_2021[3][5],r10_2021[4][5],
              r7_2022[0][5], r7_2022[1][5], r7_2022[2][5], r7_2022[3][5], r7_2022[4][5],
              r8_2022[0][5], r8_2022[1][5], r8_2022[2][5], r8_2022[3][5], r8_2022[4][5],
              r9_2022[0][5], r9_2022[1][5], r9_2022[2][5], r9_2022[3][5], r9_2022[4][5]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])

#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['CO2cc_ch', 'CO2p_ch', 'CO2cc_cp', 'CO2p_cp', 'CO2cc_hp', 'CO2p_h']
display(CoefCorrel)

,CO2cc_ch,CO2p_ch,CO2cc_cp,CO2p_cp,CO2cc_hp,CO2p_h
Sem1 Julio 2021,0.684386,1.127527e-53,0.861593,4.566303e-113,0.526225,2.222299e-28
Sem2 Julio 2021,0.758854,8.773868e-126,0.645226,1.124645e-79,0.499346,2.730589e-43
Sem3 Julio 2021,0.759351,3.670886e-127,0.629524,1.835321e-75,0.611005,5.131154e-70
Sem4 Julio 2021,0.662151,5.170644e-86,0.402246,1.592526e-27,0.543861,5.407288e-53
Sem5 Julio 2021,0.933214,2.504385e-257,0.803264,2.822684e-131,0.858662,9.394393e-169
Sem1 Agosto 2021,0.905248,7.661031e-35,0.382749,1.808597e-04,0.683664,8.144487e-14
Sem2 Agosto 2021,0.833413,1.023102e-174,0.503084,2.135026e-44,0.576420,9.339233e-61
Sem3 Agosto 2021,0.498271,4.396582e-43,0.703821,1.074882e-100,0.280488,1.662463e-13
Sem4 Agosto 2021,0.725568,7.502338e-111,0.518688,1.503308e-47,0.681027,9.546608e-93
Sem5 Agosto 2021,0.718016,5.597908e-52,0.395967,1.860053e-13,0.456366,7.241023e-18


# 2) ¿Hay relación entre la cantidad de CO2 que hay en cada sala? Coef correl CO2c-CO2h y CO2c-CO2p con las variables suavizadas

In [51]:
def CO2_entreSalas_cc_por_semanas(DataframeCruce, DataframeHall, PolicromosSemanas):
    """
    DataframeCruce, DataframeExterior, DataframeHall son todos listas de dataframes para cada semana, al igual que 
    PolicromosSemanas.
    Columnas de PolicromosSemanas= Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo
    """
    i=0
    r=[]
    while i<len(DataframeCruce):
       #print(i)
        DataframeCruce[i].reset_index(drop=True, inplace=True)
        DataframeHall[i].reset_index(drop=True, inplace=True)
        PolicromosSemanas[i].reset_index(drop=True, inplace=True)
        
        # Variables que me van a hacer falta: Tc, Te, Th, Tp, CO2c, CO2h, CO2p
        for index, _ in DataframeHall[i].iterrows():#Se va fuera de indice
            if  __builtins__.sum([pd.isna(DataframeCruce[i]['CO2c_suavizado'][index]),
                                  pd.isna(DataframeHall[i]['CO2h_suavizado'][index]),
                                  pd.isna(PolicromosSemanas[i]["CO2_suavizado"][index])]) > 0:
                DataframeCruce[i].drop([index], axis=0, inplace=True)
                DataframeHall[i].drop([index], axis=0, inplace=True) 
                PolicromosSemanas[i].drop([index], axis=0, inplace=True) 
        #Mediante scipy:
        CO2cc_ch, CO2p_ch = stats.pearsonr(DataframeCruce[i]['CO2c_suavizado'], DataframeHall[i]['CO2h_suavizado'])
        CO2cc_cp, CO2p_cp = stats.pearsonr(DataframeCruce[i]['CO2c_suavizado'], PolicromosSemanas[i]["CO2_suavizado"])
        CO2cc_hp, CO2p_hp = stats.pearsonr(DataframeHall[i]['CO2h_suavizado'], PolicromosSemanas[i]["CO2_suavizado"])
        rs= CO2cc_ch, CO2p_ch, CO2cc_cp, CO2p_cp, CO2cc_hp, CO2p_hp
        r.append(rs)
        i=i+1
    return(r)

In [52]:
#Separar por semanas los meses con oscilación clara:
#2021
r7_2021 = CO2_entreSalas_cc_por_semanas(SemanaCruceJulio21, SemanaHallJulio21, PolicromosSemanas_7_2021)
r8_2021 = CO2_entreSalas_cc_por_semanas(SemanaCruceAgosto21, SemanaHallAgosto21, PolicromosSemanas_8_2021)
r9_2021 = CO2_entreSalas_cc_por_semanas(SemanaCruceSeptiembre21, SemanaHallSeptiembre21, PolicromosSemanas_9_2021)
r10_2021 = CO2_entreSalas_cc_por_semanas(SemanaCruceOctubre21, SemanaHallOctubre21, PolicromosSemanas_10_2021)
#2022
r7_2022 = CO2_entreSalas_cc_por_semanas(SemanaCruceJulio22, SemanaHallJulio22, PolicromosSemanas_7_2022)     
r8_2022 = CO2_entreSalas_cc_por_semanas(SemanaCruceAgosto22, SemanaHallAgosto22, PolicromosSemanas_8_2022)
r9_2022 = CO2_entreSalas_cc_por_semanas(SemanaCruceSeptiembre22, SemanaHallSeptiembre22, PolicromosSemanas_9_2022)

In [53]:
s1=pd.Series([r7_2021[0][0],r7_2021[1][0], r7_2021[2][0],r7_2021[3][0],r7_2021[4][0],
              r8_2021[0][0],r8_2021[1][0], r8_2021[2][0],r8_2021[3][0],r8_2021[4][0],#semanas de agosto 21
              r9_2021[0][0],r9_2021[1][0],r9_2021[2][0],r9_2021[3][0],r9_2021[4][0],#semanas de septiembre 21
              r10_2021[0][0],r10_2021[1][0],r10_2021[2][0],r10_2021[3][0],r10_2021[4][0],
              r7_2022[0][0], r7_2022[1][0], r7_2022[2][0], r7_2022[3][0], r7_2022[4][0],  
              r8_2022[0][0], r8_2022[1][0], r8_2022[2][0], r8_2022[3][0], r8_2022[4][0],
              r9_2022[0][0], r9_2022[1][0], r9_2022[2][0], r9_2022[3][0], r9_2022[4][0]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s2=pd.Series([r7_2021[0][1],r7_2021[1][1], r7_2021[2][1],r7_2021[3][1],r7_2021[4][1],
              r8_2021[0][1],r8_2021[1][1], r8_2021[2][1],r8_2021[3][1],r8_2021[4][1],#semanas de agosto 21
              r9_2021[0][1],r9_2021[1][1],r9_2021[2][1],r9_2021[3][1],r9_2021[4][1],#semanas de septiembre 21
              r10_2021[0][1],r10_2021[1][1],r10_2021[2][1],r10_2021[3][1],r10_2021[4][1],
              r7_2022[0][1], r7_2022[1][1], r7_2022[2][1], r7_2022[3][1], r7_2022[4][1],  
              r8_2022[0][1], r8_2022[1][1], r8_2022[2][1], r8_2022[3][1], r8_2022[4][1],
              r9_2022[0][1], r9_2022[1][1], r9_2022[2][1], r9_2022[3][1], r9_2022[4][1]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s3=pd.Series([r7_2021[0][2],r7_2021[1][2], r7_2021[2][2],r7_2021[3][2],r7_2021[4][2],
              r8_2021[0][2],r8_2021[1][2], r8_2021[2][2],r8_2021[3][2],r8_2021[4][2],#semanas de agosto 21
              r9_2021[0][2],r9_2021[1][2],r9_2021[2][2],r9_2021[3][2],r9_2021[4][2],#semanas de septiembre 21
              r10_2021[0][2],r10_2021[1][2],r10_2021[2][2],r10_2021[3][2],r10_2021[4][2],
              r7_2022[0][2], r7_2022[1][2], r7_2022[2][2], r7_2022[3][2], r7_2022[4][2], 
              r8_2022[0][2], r8_2022[1][2], r8_2022[2][2], r8_2022[3][2], r8_2022[4][2],
              r9_2022[0][2], r9_2022[1][2], r9_2022[2][2], r9_2022[3][2], r9_2022[4][2]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s4=pd.Series([r7_2021[0][3],r7_2021[1][3], r7_2021[2][3],r7_2021[3][3],r7_2021[4][3],
              r8_2021[0][3],r8_2021[1][3], r8_2021[2][3],r8_2021[3][3],r8_2021[4][3],#semanas de agosto 21
              r9_2021[0][3],r9_2021[1][3],r9_2021[2][3],r9_2021[3][3],r9_2021[4][3],
              r10_2021[0][3],r10_2021[1][3], r10_2021[2][3],r10_2021[3][3],r10_2021[4][3],
              r7_2022[0][3], r7_2022[1][3], r7_2022[2][3], r7_2022[3][3], r7_2022[4][3],
              r8_2022[0][3], r8_2022[1][3], r8_2022[2][3], r8_2022[3][3], r8_2022[4][3],
              r9_2022[0][3], r9_2022[1][3], r9_2022[2][3], r9_2022[3][3], r9_2022[4][3]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s5=pd.Series([r7_2021[0][4],r7_2021[1][4], r7_2021[2][4],r7_2021[3][4],r7_2021[4][4],
              r8_2021[0][4],r8_2021[1][4], r8_2021[2][4],r8_2021[3][4],r8_2021[4][4],#semanas de agosto 21
              r9_2021[0][4],r9_2021[1][4],r9_2021[2][4],r9_2021[3][4],r9_2021[4][4],#semanas de septiembre 21
              r10_2021[0][4],r10_2021[1][4],r10_2021[2][4],r10_2021[3][4],r10_2021[4][4],
              r7_2022[0][4], r7_2022[1][4], r7_2022[2][4], r7_2022[3][4], r7_2022[4][4], 
              r8_2022[0][4], r8_2022[1][4], r8_2022[2][4], r8_2022[3][4], r8_2022[4][4],
              r9_2022[0][4], r9_2022[1][4], r9_2022[2][4], r9_2022[3][4], r9_2022[4][4]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])
s6=pd.Series([r7_2021[0][5],r7_2021[1][5], r7_2021[2][5],r7_2021[3][5],r7_2021[4][5],
              r8_2021[0][5],r8_2021[1][5], r8_2021[2][5],r8_2021[3][5],r8_2021[4][5],#semanas de agosto 21
              r9_2021[0][5],r9_2021[1][5],r9_2021[2][5],r9_2021[3][5],r9_2021[4][5],#semanas de septiembre 21
              r10_2021[0][5],r10_2021[1][5],r10_2021[2][5],r10_2021[3][5],r10_2021[4][5],
              r7_2022[0][5], r7_2022[1][5], r7_2022[2][5], r7_2022[3][5], r7_2022[4][5],
              r8_2022[0][5], r8_2022[1][5], r8_2022[2][5], r8_2022[3][5], r8_2022[4][5],
              r9_2022[0][5], r9_2022[1][5], r9_2022[2][5], r9_2022[3][5], r9_2022[4][5]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021','Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022','Sem5 Septiembre 2022'])

#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['CO2cc_ch', 'CO2p_ch', 'CO2cc_cp', 'CO2p_cp', 'CO2cc_hp', 'CO2p_h']
display(CoefCorrel)

,CO2cc_ch,CO2p_ch,CO2cc_cp,CO2p_cp,CO2cc_hp,CO2p_h
Sem1 Julio 2021,0.702115,1.463921e-57,0.898622,5.825606e-137,0.571443,3.090859e-34
Sem2 Julio 2021,0.766688,6.570748e-130,0.692617,2.757935e-96,0.548156,1.678592e-53
Sem3 Julio 2021,0.765474,2.075223e-130,0.686786,6.654229e-95,0.662607,3.604637e-86
Sem4 Julio 2021,0.672153,1.613731e-89,0.480144,4.762673e-40,0.592026,8.521261e-65
Sem5 Julio 2021,0.935498,1.613893e-261,0.836649,3.475856e-152,0.885475,3.349004e-193
Sem1 Agosto 2021,0.912490,2.616027e-36,0.507882,2.766164e-07,0.764112,1.245556e-18
Sem2 Agosto 2021,0.838832,4.216205e-179,0.576207,1.056390e-60,0.642024,2.362518e-79
Sem3 Agosto 2021,0.496760,8.560411e-43,0.761800,2.575607e-127,0.310414,2.420104e-16
Sem4 Agosto 2021,0.736609,6.644533e-116,0.594207,2.229316e-65,0.724441,2.384302e-110
Sem5 Agosto 2021,0.741719,4.142529e-57,0.494203,4.127616e-21,0.537727,2.234407e-25
